In [ ]:
import torch
import torch.onnx
from ultralytics import YOLO

# 모델 로드
model = YOLO(r"D:\competition\yolo-V8\runs\detect\train\weights\best.pt")
model.to('cuda')

# 데이터셋 경로
# data_path = r"C:\code\nextchip.yaml"

# 모델을 평가 모드로 설정
model.eval()

# 양자화 준비
model.qconfig = torch.quantization.get_default_qconfig('fbgemm')
torch.quantization.prepare(model.model, inplace=True)

# 캘리브레이션 데이터로 모델 실행
dummy_input = torch.randn(1, 3, 640, 640).to('cuda')
model.model(dummy_input)

# 모델 양자화
torch.quantization.convert(model.model, inplace=True)

# 양자화된 모델을 ONNX로 내보내기 (ONNX 변환 시 모델은 CPU로 이동 필요)
dummy_input_cpu = dummy_input.to('cpu')
model.model.to('cpu')  # ONNX로 내보낼 때는 CPU로 모델 전환

# 양자화된 모델을 ONNX로 내보내기
torch.onnx.export(model.model,
                  dummy_input_cpu,
                  r"D:\competition\yolov8_onnx\yolov8s_quantinized.onnx",
                  export_params=True)


In [ ]:
import onnx
from onnxruntime.quantization import quantize_dynamic, QuantType

# ONNX 모델 로드
onnx_model_path = r"C:\code\onnx-modifier\modified_onnx\modified_yolov5s_simplified.onnx"
quantized_model_path = r"C:\code\yolov5s_onnx\yolov5s_quantized.onnx"

# 양자화 적용 (INT8)
quantize_dynamic(onnx_model_path, quantized_model_path, weight_type=QuantType.QInt8)

print(f"양자화된 모델이 저장되었습니다: {quantized_model_path}")

In [ ]:
import onnx
from onnx import helper, shape_inference

# ONNX 모델 로드
onnx_model_path = r"C:\code\yolov8s_onnx\yolov8s.onnx"
modified_model_path = r"C:\code\yolov8s_onnx\modified_yolov8s.onnx"

# 모델 로드
model = onnx.load(onnx_model_path)

# ConvInteger와 DynamicQuantizeLinear 제거
nodes_to_remove = []
for node in model.graph.node:
    if node.op_type in ["ConvInteger", "DynamicQuantizeLinear"]:
        nodes_to_remove.append(node)

# 모델 그래프에서 해당 노드 제거
for node in nodes_to_remove:
    model.graph.node.remove(node)

# 모델에 다른 필요한 변경이 있으면 여기에서 수행

# 모델 저장 전 shape_inference 수행
model = shape_inference.infer_shapes(model)

# 수정된 모델 저장
onnx.save(model, modified_model_path)

print(f"수정된 모델이 저장되었습니다: {modified_model_path}")


In [ ]:
# import torch
# import torch.nn as nn
# import torch.quantization
# import torchvision.transforms as transforms
# import torchvision.datasets as datasets
# from torch.utils.data import DataLoader
# from ultralytics import YOLO

# # 1. 이미 학습된 모델 로드
# model = YOLO(r'C:\code\yolov8_onnx\best.pt')  # 학습된 모델 로드

# # 새로운 YAML 파일 경로 설정
# new_yaml_path = r"C:\code\nextchip.yaml"
# model.overrides['data'] = new_yaml_path

# model.eval()  # 평가 모드로 전환

# # 2. 모델 양자화 준비
# model.qconfig = torch.quantization.get_default_qconfig('fbgemm')  # 또는 'qnnpack'
# torch.quantization.prepare(model, inplace=True)

# # 3. 성능 비교를 위한 데이터 준비
# # CIFAR-10 데이터셋을 사용하여 테스트 데이터셋 준비
# transform = transforms.Compose([
#     transforms.Resize((32, 32)),  # 모델 입력 크기에 맞게 조정
#     transforms.ToTensor(),  # Tensor로 변환
# ])
# test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
# test_loader = DataLoader(test_dataset, batch_size=10, shuffle=False)

# # 4. 원본 모델 성능 평가
# original_correct = 0
# total = 0

# with torch.no_grad():
#     for images, labels in test_loader:
#         original_output = model(images)  # 원본 모델 예측
#         _, predicted = torch.max(original_output.data, 1)
#         total += labels.size(0)
#         original_correct += (predicted == labels).sum().item()

# # 정확도 계산
# original_accuracy = 100 * original_correct / total
# print(f"Original Model Accuracy: {original_accuracy:.2f}%")

# # 5. 양자화
# torch.quantization.convert(model, inplace=True)

# # 6. 양자화된 모델 성능 평가
# quantized_correct = 0
# total = 0

# with torch.no_grad():
#     for images, labels in test_loader:
#         quantized_output = model(images)  # 양자화된 모델 예측
#         _, predicted = torch.max(quantized_output.data, 1)
#         total += labels.size(0)
#         quantized_correct += (predicted == labels).sum().item()

# # 정확도 계산
# quantized_accuracy = 100 * quantized_correct / total
# print(f"Quantized Model Accuracy: {quantized_accuracy:.2f}%")

# # 7. 성능 비교 출력
# print(f"Original Model Accuracy: {original_accuracy:.2f}%")
# print(f"Quantized Model Accuracy: {quantized_accuracy:.2f}%")


In [ ]:
import torch
from ultralytics import YOLO

# PyTorch 모델 로드
model = YOLO(r"C:\code\yolov8_onnx\yolov8s.pt")

# 평가 모드로 설정
model.eval()

# 더미 입력 생성 (ONNX 변환을 위한 입력 샘플)
dummy_input = torch.randn(1, 3, 640, 640)

# PyTorch 모델을 ONNX 형식으로 내보내기
onnx_model_path = r"C:\code\yolov8s_onnx\yolov8s.onnx"
torch.onnx.export(model.model,  # PyTorch 모델 내부 구조를 넘겨줘야 함
                  dummy_input,
                  onnx_model_path,
                  export_params=True,
                  opset_version=13,
                  do_constant_folding=True,  # 상수 폴딩 최적화
                  input_names=["input"],
                  output_names=["output"])

print(f"ONNX 모델이 저장되었습니다: {onnx_model_path}")
